In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from collections import defaultdict


num_class = 256

Using TensorFlow backend.


In [25]:
import keras
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.optimizers import SGD
from keras.initializations import normal


def getModel( output_dim ):
    ''' 
        * output_dim: the number of classes (int)
        * return: compiled model (keras.engine.training.Model)
    '''
    vgg_model = VGG16( weights='imagenet', include_top=True )
    vgg_out = vgg_model.layers[-2].output #Last FC layer's output  
    softmax_layer = Dense(output_dim, activation='softmax')(vgg_out) #Create softmax layer taking input as vgg_out
    #Create new transfer learning model
    tl_model = Model( input=vgg_model.input, output=softmax_layer )

    #Freeze all layers of VGG16 and Compile the model
    for layer in vgg_model.layers:
        layer.trainable = False
    #Confirm the model is appropriate
    return tl_model

model = getModel(num_class)
# sgd = SGD(lr=1e-3, decay=1e-4, momentum=0.9, nesterov=True)
# model.compile(optimizer='rmsprop',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

In [3]:
def list_gen():
    root_dir = '../256_ObjectCategories/'
    image_dirs = glob.glob(root_dir + '*')
    d = defaultdict(list)
    for idir in image_dirs:
        all_images = glob.glob(idir + '/*.jpg')
        label_id = int(idir.split('/')[-1].split('.')[0])-1
        if label_id > 255:
            continue
        np.random.shuffle(all_images)
        train_paths, val_paths, test_paths = all_images[:len(all_images)/10*8], all_images[len(all_images)/10*8:len(all_images)/10*9], all_images[len(all_images)/10*9:]
        train_labels, val_labels, test_labels = [label_id]*(len(all_images)/10*8), [label_id]*(len(all_images)/10), [label_id]*(len(all_images)/10)
        
        d['train_images'].append(train_paths)
        d['val_images'].append(val_paths)
        d['test_images'].append(test_paths)
        d['train_labels'].append(train_labels)
        d['val_labels'].append(val_labels)
        d['test_labels'].append(test_labels)
    return d

all_d = list_gen()

In [4]:
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import np_utils
from keras.applications.imagenet_utils import preprocess_input

def load_data(image_paths, labels, crop_size):
    X = np.zeros((len(image_paths), crop_size, crop_size, 3))
    for i,path in enumerate(image_paths):
        X[i, :] = img_to_array(load_img(path, target_size=(crop_size, crop_size)))
    y = np_utils.to_categorical(labels)
    return X, y

In [34]:
crop_size = 224
all_per_class = 16
train_paths = []
val_paths = []
test_paths = []
train_labels = []
val_labels = []
test_labels = []
for i in xrange(num_class):
    train_paths += (all_d['train_images'][i][:all_per_class])
    val_paths += (all_d['val_images'][i][:all_per_class])
    test_paths += (all_d['test_images'][i][:all_per_class])
    train_labels += (all_d['train_labels'][i][:all_per_class])
    val_labels += (all_d['val_labels'][i][:all_per_class])
    test_labels += (all_d['test_labels'][i][:all_per_class])

X_train, y_train = load_data(train_paths, train_labels, crop_size)
X_val, y_val = load_data(val_paths, val_labels, crop_size)
X_test, y_test = load_data(test_paths, test_labels, crop_size)

In [35]:
X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)

In [36]:
datagen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
    rescale=1./128)

datagen.fit(X_train)

In [ ]:
for sample_per_class in [2,4,8,16]:
    train = X_train.reshape((num_class, -1, crop_size, crop_size, 3))
    train = train[:, :sample_per_class].reshape((-1, crop_size, crop_size, 3))
    label = y_train.reshape((num_class, -1, num_class))
    label = label[:, :sample_per_class].reshape((-1, num_class))
    rmsmodel = getModel(num_class)
    rmsmodel.compile(optimizer='rmsprop',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])
    rmsmodel.fit_generator(datagen.flow(train, label, batch_size=32),
                           validation_data=datagen.flow(X_val, y_val, batch_size=32),
                           nb_val_samples=200,
                           samples_per_epoch=len(train), nb_epoch=40)

Epoch 1/40
512/512 [==============================] - 214s - loss: 9.3182 - acc: 0.0000e+00 - val_loss: 7.1529 - val_acc: 0.0179
Epoch 2/40
512/512 [==============================] - 219s - loss: 6.6413 - acc: 0.0059 - val_loss: 6.1498 - val_acc: 0.0089
Epoch 3/40
512/512 [==============================] - 219s - loss: 5.7114 - acc: 0.0234 - val_loss: 5.9294 - val_acc: 0.0312
Epoch 4/40
512/512 [==============================] - 221s - loss: 5.0933 - acc: 0.0371 - val_loss: 5.7414 - val_acc: 0.0446
Epoch 5/40
512/512 [==============================] - 221s - loss: 4.4736 - acc: 0.0762 - val_loss: 5.4127 - val_acc: 0.0491
Epoch 6/40
512/512 [==============================] - 221s - loss: 3.8902 - acc: 0.1172 - val_loss: 5.3540 - val_acc: 0.0536
Epoch 7/40
512/512 [==============================] - 221s - loss: 3.3639 - acc: 0.1855 - val_loss: 4.9833 - val_acc: 0.1071
Epoch 8/40
512/512 [==============================] - 221s - loss: 2.9045 - acc: 0.2734 - val_loss: 4.7824 - val_acc: 0.1

In [26]:
rmsmodel = getModel(num_class)
# sgd = SGD(lr=1e-3, decay=1e-4, momentum=0.9, nesterov=True)
rmsmodel.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
rmsmodel.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                    samples_per_epoch=len(X_train), nb_epoch=10)

Epoch 1/10
1024/1024 [==============================] - 302s - loss: 8.6288 - acc: 0.0088   
Epoch 2/10
1024/1024 [==============================] - 304s - loss: 6.0655 - acc: 0.0332   
Epoch 3/10
1024/1024 [==============================] - 304s - loss: 5.1832 - acc: 0.0654   
Epoch 4/10
1024/1024 [==============================] - 304s - loss: 4.4927 - acc: 0.1006   
Epoch 5/10
1024/1024 [==============================] - 305s - loss: 3.8219 - acc: 0.1445   
Epoch 6/10
1024/1024 [==============================] - 306s - loss: 3.3476 - acc: 0.2168   
Epoch 7/10
1024/1024 [==============================] - 312s - loss: 2.8463 - acc: 0.2979   
Epoch 8/10
 384/1024 [==========>...................] - ETA: 193s - loss: 2.6298 - acc: 0.3047

KeyboardInterrupt: 